# Tensorflow

Tensorflow (and PyTorch) are both Python packages for doing Deep Learning, specifically modeling phenomena with neural networks.

For this notebook, grateful acknowledgement is made to:
https://machinelearningmastery.com/tensorflow-tutorial-deep-learning-with-tf-keras/

## First, a little digression on neural networks:

<img src='data-sci-images/iris-nn.png' width=500>
[Image taken from https://user-images.githubusercontent.com/35667308/48126015-2354ab00-e2a6-11e8-9a22-c58ad6ee7733.png]

## The basic idea behind neural networks:

* Each data point can be "propagated" forward through the network based on numerical weights attached to each node.
* At the end, the error in the final result is calculated based on some criterion for measuring errors
* The layers are then traversed in reverse order, and it is mathematically determined how the weights might be slightly tweaked so as to give less error the next time around
* This happens many times until ideally the errors decrease and the accuracy of predictions is thought to be sound

# Import the libraries

In [ ]:
# mlp for multiclass classification
from numpy import argmax
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
# load the dataset

# Get the data

In [ ]:
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv'
df = read_csv(path, header=None)

In [ ]:
# split into input and output columns
X = df.values[:, :-1]
y = df.values[:, -1]

In [ ]:
X

In [ ]:
X.dtype

In [ ]:
# ensure all data are floating point values
X = X.astype('float32')

In [ ]:
y

In [ ]:
# encode strings to integer
le = LabelEncoder() 
y = le.fit_transform(y)

In [ ]:
le.inverse_transform([0])

In [ ]:
y

# Use 2/3 of the data for training, and reserve 1/3 for testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Check number of input features

In [ ]:
n_features = X_train.shape[1]

In [ ]:
n_features

# Define the model

* input_shape for the input layer has to match the number of features
* activation on the output layer is commonly softmax for classification, as that returns values in the range (0,1), and values sum to 1. 

In [ ]:
model = Sequential()
model.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(3, activation='softmax'))

# Define the methods for assessing error levels during training

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Keep track of information during each epoch for later assessment

In [ ]:
train_loss_results = []
train_accuracy_results = []

class CustomCallback(keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        train_loss_results.append(logs['loss'])
        train_accuracy_results.append(logs['accuracy'])
        
        if epoch % 30 == 0:
            keys = list(logs.keys())
            #print("End epoch {} of training; got log keys: {}".format(epoch, keys))
            print("Acc: {}, Loss: {}".format(logs['accuracy'], logs['loss']))

# Train the model

* epoch count is the number of passes through the entire data set during training
* batch_size is the number of data points to work through before updating the model's internal parameters

In [ ]:
model.fit(X_train, y_train, epochs=150, batch_size=32, verbose=0, callbacks=[CustomCallback()])

# Evaluate the model

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.3f' % acc)

# Look at the evolution of Loss and Accuracy during training

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))
fig.suptitle('Training Metrics')

axes[0].set_ylabel("Loss", fontsize=14)
axes[0].plot(train_loss_results)

axes[1].set_ylabel("Accuracy", fontsize=14)
axes[1].set_xlabel("Epoch", fontsize=14)
axes[1].plot(train_accuracy_results)
plt.show()

# Use the model to make predictions

In [ ]:
row = [5.1,3.5,1.4,0.2]
yhat = model.predict([row])
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

In [ ]:
row = [5.1,3.5,1.4,0.2]
yhat = model.predict([row])
print('Predicted: %s \nClass: %s' % (yhat, le.inverse_transform([argmax(yhat)])))